In [1]:
import cvxpy as cp
import numpy as np
import scipy
import scipy.sparse as sparse
import matplotlib.pyplot as plt

np.random.seed(0)

# Example from the documentation

In [2]:
# Generate a random SDP.
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())
sing_eigval = np.array([0., 0., 2.])
_, sing_eigvec = np.linalg.eig(np.random.random((n, n)))
A.append(np.random.random((n,n)))
# If the whole quad matrix is singular dont work
#A.append(sing_eigvec@np.diag(sing_eigval)@sing_eigvec.conj().T)
b.append(np.random.randn())
# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p + 1)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)), constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)


The optimal value is 2.7064903389012436
A solution X is
[[ 1.40057161 -0.47647128 -0.61079777]
 [-0.47647128  0.19268429  0.04784566]
 [-0.61079777  0.04784566  1.10226263]]


# Small dimension SDR

In [3]:
N_omega = 5
n = 2
obj_mat = np.zeros((4*N_omega, 4*N_omega))
obj_mat[:N_omega, :N_omega] = -np.eye(N_omega)
obj_mat[3*N_omega:, 3*N_omega:] = (n**2/N_omega)*np.eye(N_omega)
constr_quad_mat = np.zeros((4*N_omega, 4*N_omega))
constr_quad_mat[:N_omega, :N_omega] = np.eye(N_omega)
constr_quad_mat[3*N_omega:, 3*N_omega:] = -(n**2/N_omega)*np.eye(N_omega)
P_J = np.eye(N_omega)
P_J.resize((3*N_omega, N_omega))
constr_lin_mat = np.block([[np.zeros((3*N_omega, 3*N_omega)), 0.5*P_J],[0.5*P_J.conj().T, (-n/N_omega)*np.eye(N_omega)]])
def_J_mat = np.zeros((3*N_omega, 3*N_omega))
def_J_mat[N_omega:, N_omega:] = np.eye(2*N_omega)
def_J_mat = np.block([[def_J_mat, -2*P_J],[-2*P_J.conj().T, -2*np.eye(N_omega)]])
N_matrices = []
kron_delta = []
for i in range(N_omega):
    for j in range(i + 1):
        mat = np.zeros((N_omega, N_omega))
        mat[i, j] = 1
        N_matrices.append(np.block([[np.zeros((3*N_omega, 3*N_omega)), np.zeros((3*N_omega, N_omega))],[np.zeros((N_omega, 3*N_omega)), mat + mat.T]]))
        if i == j:
            kron_delta.append(2)
        else:
            kron_delta.append(0)

In [4]:
X = cp.Variable((4*N_omega, 4*N_omega), symmetric = True)
constraints = [X >> 0]
constraints += [cp.trace(constr_quad_mat@X) == 0, cp.trace(constr_lin_mat@X) == 0, cp.trace(def_J_mat@X) == 0]
constraints += [cp.trace(N_matrices[i]@X) == kron_delta[i] for i in range(len(N_matrices))]
problem = cp.Problem(cp.Minimize(cp.trace(obj_mat @ X)), constraints)
problem.solve(verbose = True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Feb 02 10:20:20 AM: Your problem has 400 variables, 19 constraints, and 0 parameters.
(CVXPY) Feb 02 10:20:20 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 02 10:20:20 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 02 10:20:20 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 02 10:20:20 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 02 10:20:20 AM: Compiling problem (target solver=SCS).
(CV

8.215855329396504e-09

In [76]:
constraints[4].dual_value

-1.1509095456723938e-07

In [51]:
A = np.random.random((N_omega, N_omega))
A = A@A.conj().T
eig_val, sing_eigvec = np.linalg.eig(A)
Z = sing_eigvec@sing_eigvec.conj().T
beta = 0.1*scipy.linalg.hankel(np.random.random(N_omega), np.random.random(N_omega))
W_plus = scipy.linalg.expm(beta)
W_minus = scipy.linalg.expm(beta)
J = 0.25*(W_plus@W_plus.conj().T + W_minus@W_minus.conj().T - 2*np.eye(N_omega))
Y = np.concatenate([J, W_plus, W_minus, Z])
X = Y@Y.conj().T

In [42]:
Q = def_J_mat[:3*N_omega, :3*N_omega]
P = def_J_mat[:3*N_omega, 3*N_omega:]

In [44]:
primal = np.concatenate([J, W_plus, W_minus])
np.trace(primal.conj().T@Q@primal + Z.conj().T@P.conj().T@primal + primal.conj().T@P@Z + Z.conj().T@def_J_mat[3*N_omega:, 3*N_omega:]@Z), np.trace(def_J_mat@X)

(-2.220446049250313e-16, -2.6645352591003757e-15)

In [55]:
np.array([np.trace(N_matrices[i]@X) for i in range(len(N_matrices))]) + kron_delta

array([-1.77635684e-15,  1.71824360e-15, -2.22044605e-15, -7.29884903e-16,
        1.55431223e-15,  3.55271368e-15, -1.01633112e-15, -2.77555756e-15,
        2.22044605e-16,  8.88178420e-16,  8.95550994e-17,  4.44089210e-16,
       -8.88178420e-16,  7.77156117e-16, -8.88178420e-16])

In [130]:
np.trace(primal.conj().T@Q@primal + Z.conj().T@P.conj().T@primal + primal.conj().T@P@Z) + n**2

-3.673320429906303